In [1]:
import sys
from pathlib import Path
import numpy as np

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))


import tensorflow as tf
from tensorflow.keras.models import model_from_json

from model.model import Model

%load_ext autoreload
%autoreload 2

2026-01-27 21:28:56.253612: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-27 21:28:56.254712: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-27 21:28:56.340313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-27 21:28:57.766955: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

2.20.0


In [2]:
artifacts_dir = "data"

In [ ]:
with open(f"{artifacts_dir}/mod_unet_last_weights_v5.json", "r") as f:
    model = model_from_json(f.read())

model.load_weights(f"{artifacts_dir}/mod_unet_last_weights_v5.weights.h5")

2026-01-27 21:28:59.205760: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/misha/repo/Cassette-denoising/venv_tf2/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 50 variables whereas the saved optimizer has 98 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
denoiser = Model(
    model=model,
    weights=None,
    model_name="unet_denoiser",
    sample_rate=44100,
    min_duration=1.0,
    frame_length=262144,  # 512x512 model uses larger frames
    hop_length_frame=262144,  # No overlap (full frame hops)
    n_fft=1023,  # 512x512 model uses larger FFT
    hop_length_fft=512
)

In [ ]:
denoiser.predict(artifacts_dir, ["ELO_1_raw.wav"], "third_test.wav", 
                  a_in=40.0, b_in=40.0, a_ou=40.0, b_ou=40.0)

128


2026-01-27 21:30:42.674290: I external/local_xla/xla/service/service.cc:163] XLA service 0x7bb43400eb40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2026-01-27 21:30:42.674372: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Host, Default Version
2026-01-27 21:30:42.707329: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769545842.789280  727597 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step
(401, 128, 128)
(401, 128, 128)
32768
256


In [ ]:
# model = tf.keras.models.load_model(f"{artifacts_dir}/model_best.karas", compile=False)